In [ ]:
from fastai.vision.all import *
import torchvision 
import numpy as np
import torch
import cv2
import os
from modules_th.video_block import *
import random
import time

In [ ]:
mean = [0.43216, 0.394666, 0.37645]
std = [0.22803, 0.22145, 0.216989]

model = create_body(torchvision.models.video.r2plus1d_18, cut=-2)

In [ ]:
path_df = '/mnt/data/eugeniomarinelli/UCF_experiments/train_UCF.csv'
df = pd.read_csv('/mnt/data/eugeniomarinelli/UCF_experiments/train_UCF.csv', index_col=0)
df.head(1)

,vid_files,lbls,val,embs_path
0,/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Testing_Normal_Videos_Anomaly/Normal_Videos_903_x264.mp4,Testing_Normal_Videos_Anomaly,False,/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Testing_Normal_Videos_Anomaly/Normal_Videos_903_x264


In [ ]:
vid_paths = df.vid_files.values

In [ ]:
duration

2884.0

In [ ]:
def add_frame_to_snip(res, frame, snip, j):
    if res: 
        snip[j] = tensor(frame)
        j += 1
    return snip, j

def snip_is_ready(j, sl):
    return j >= sl

def create_and_save_embs(snip, pl, embs_path, k):
    emb = model(pl(snip)[None])
#     torch.save(emb, embs_path/f'{embs_path.name}_{k}.pt') # save embeddings  
    return k + 1

def get_embs_of_video(vid_path, sl, size):    
    vidcap = cv2.VideoCapture(str(random.choice(vid_paths)))
    duration = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    with torch.no_grad():
        n_loaded_frames, n_created_embs = 0, 0  # j counts # of frames loaded mod sl, k counts # of snippets
        snip = torch.zeros((sl,240,320,3), dtype=torch.uint8)
        for i in range(duration):
            if duration - i < sl: break
            res, frame = vidcap.read()
            snip, j = add_frame_to_snip(res, frame, snip, j)
            if snip_is_ready(j, sl): 
                k = create_and_save_embs(snip, pl, '', k)
                j = 0

    vidcap.release()

In [ ]:
sl, size = 16, 60, 224
pl = Pipeline([Video.create, 
               Resize(size,method=ResizeMethod.Pad), 
               ToTensor(), 
               IntToFloatTensor(), 
               Normalize.from_stats(mean,std)])



221.89058896782808

In [ ]:
440/60

7.333333333333333

In [ ]:
duration/sl/bs

1.8697916666666667

In [ ]:
bs, sl, size = 16, 60, 224
pl = Pipeline([Video.create, 
               Resize(size,method=ResizeMethod.Pad), 
               ToTensor(), 
               IntToFloatTensor(), 
               Normalize.from_stats(mean,std)])

for j, path in enumerate(vid_paths):
#        if isinstance(df.embs_path.values[j],str) : continue; # start where we left
    print(j, end='\r')
    embs_path = Path(path[:-4]) # path to save emb video snippets 
    if not os.path.isdir(embs_path): os.mkdir(embs_path) # make folder

    vidcap = cv2.VideoCapture(path)
    duration = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

    with torch.no_grad():
        j, k = 0, 0  # j counts # of frames loaded mod sl, k counts # of snippets
        snip = torch.zeros((sl,240,320,3), dtype=torch.uint8)
        for i in range(duration):
            if duration - i < sl: break
            res, frame = vidcap.read()
            snip, j = add_frame_to_snip(res, frame, snip, j)
            if snip_is_ready(j, sl): 
                k = create_and_save_embs(snip, pl, '', k)
                j = 0

    vidcap.release()
            df.embs_path[j] = str(embs_path)
            df.to_csv(path_df)

<ipython-input-26-ea47d2630e30>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.embs_path[j] = str(embs_path)


mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at

In [ ]:
sl = 32
bs = 8
int2f = IntToFloatTensor()
pl = Pipeline([Resize(224, method=ResizeMethod.Pad),ToTensor(),IntToFloatTensor(), Normalize.from_stats(mean,std)])
with torch.no_grad(): 
    for j, path in enumerate(vid_paths):
#        if isinstance(df.embs_path.values[j],str) : continue; # start where we left
        print(j, end='\r')
        embs_path = Path(path[:-4]) # path to save emb video snippets 
        if not os.path.isdir(embs_path): os.mkdir(embs_path) # make folder
        times = L(L(), L(), L())
        i = 0
        for start in range(0, int(duration)-sl*bs, sl*bs):
            l = duration - duration%(sl*bs) if duration < sl*bs else sl*bs
            
            t0 = time.perf_counter()
            snips = create_video(path, start = start, l=l, skip=1, form='img')
            times[0].append(time.perf_counter()-t0)
            
            snips = L(chunked(snips, sl)).map(Video).map(pl).stack()       
            
            t1 = time.perf_counter()
            embs = model(snips.cuda())
            times[1].append(time.perf_counter()-t1)
            
            t2 = time.perf_counter()
            for emb in embs:
                torch.save(emb, embs_path/f'{embs_path.name}_{i}.pt') # save embeddings
                i +=1
            times[2].append(time.perf_counter()-t2)
            
        print(embs_path, end='\r')
        df.embs_path[j] = str(embs_path)
        df.to_csv(path_df)

<ipython-input-26-ea47d2630e30>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.embs_path[j] = str(embs_path)


mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at 13 9
error while decoding MB 13 9
mb_type 61 in P slice too large at

In [ ]:
get_files('/mnt/data/eugeniomarinelli/UCF_Crimes/Video', extensions='.pt')

'Normal_Videos437_x264.'

In [ ]:
L(chunked(snips, sl)).map(Video).map(pl).stack().shape

(16, 3, 32, 224, 224)

In [ ]:
files = get_files('/mnt/data/eugeniomarinelli/UCF_Crimes/Videos',extensions='.pt')
files.map(os.remove)

(#76600) [None,None,None,None,None,None,None,None,None,None...]